In [7]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import numpy as np
import re
import pandas as pd

def get_data(url):
    '''Function that returns urls for books from one page in a category'''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    product_links = soup.find_all('article', class_ = 'product_pod')
    link_array = []
    for link in product_links:
        link = link.div.a['href']
        link_array.append("/".join(url.split("/")[:-1]) + "/" + link)

    #print(link_array)
    return link_array

get_data('http://books.toscrape.com/index.html')

['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'http://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
 'http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'http://books.toscrape.com/catalogue/the-black-maria_991/index.html',
 'http://books.toscrape.com/catalogue/starving-hearts-triangular-trade-trilogy-1_990

In [8]:
def pages(url):
    '''Some categories have more than 1 page. This function gets the next pages'''
    new_page = url
    pages_urls = []
    #if page exists its status is 200
    while requests.get(new_page).status_code == 200:
        pages_urls.append(new_page)
        if len(pages_urls) == 1:
            new_page = pages_urls[-1].split("index")[0] + 'page' + '-'+ str(2) + ".html"
        else:
            new_page = new_page.split('page')[0] + 'page-' + str(int(new_page.split('page')[1].split('-')[1][:-5]) + 1) +'.html'
    return pages_urls

In [9]:
#check it works correctly
pages('http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html')

['http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html',
 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html',
 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html']

In [16]:
main_url2 = "http://books.toscrape.com/"


def main2():
    response = requests.get('http://books.toscrape.com/index.html')
    soup = BeautifulSoup(response.text, 'html.parser')
    url_array = []
    urls = soup.find_all('a', href=re.compile("catalogue/category/books"))[1:]
    for url in urls:
        url = url['href']
        url_array.append(main_url2 + url)

    all_books = []
    for i in url_array:
        all_pages = pages(i)
        all_books.append(all_pages)
    all_books = np.concatenate(all_books)
    #print(all_books)
    rf = []
    for url in all_books:
        books_urls = get_data(url)
        #print(books_urls)
        
        
        for url2 in books_urls:
            response2 = requests.get(url2)
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            title = soup2.find('div',class_ = 'col-sm-6 product_main').h1.get_text()
            price = soup2.find('p',class_ = 'price_color').get_text()[1:]
            category = soup2.find_all('a')[3].get_text()
            rating = soup2.find('p', class_ = 'star-rating')['class'][1]
            
            result = [title, category, rating, price]
           
            rf.append(result)
            
    #print(rf)
    return rf

In [17]:
#k = np.array(main2())
#print(k)
main2()


[["It's Only the Himalayas", 'Travel', 'Two', '£45.17'],
 ['Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  'Travel',
  'Four',
  '£49.43'],
 ['See America: A Celebration of Our National Parks & Treasured Sites',
  'Travel',
  'Three',
  '£48.87'],
 ['Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  'Travel',
  'Two',
  '£36.94'],
 ['Under the Tuscan Sun', 'Travel', 'Three', '£37.33'],
 ['A Summer In Europe', 'Travel', 'Two', '£44.34'],
 ['The Great Railway Bazaar', 'Travel', 'One', '£30.54'],
 ['A Year in Provence (Provence #1)', 'Travel', 'Four', '£56.88'],
 ['The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
  'Travel',
  'One',
  '£23.21'],
 ['Neither Here nor There: Travels in Europe', 'Travel', 'Three', '£38.95'],
 ['1,000 Places to See Before You Die', 'Travel', 'Five', '£26.08'],
 ['Sharp Objects', 'Mystery', 'Four', '£47.82'],
 ['In a Dark, Dark Wood', 'Mystery', 'One', '£19.6

In [18]:
#main2()
n = 0
frames = []

for i in main2():
    n +=1
    df_data = pd.DataFrame({'name': i[0], 'genre': i[1], 'rating (out of 5)': i[2], 'price': i[3]}, index = [n])
    frames.append(df_data)
    
final_df = pd.concat(frames)

In [19]:
final_df

,name,genre,rating (out of 5),price
1,It's Only the Himalayas,Travel,Two,£45.17
2,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,Four,£49.43
3,See America: A Celebration of Our National Par...,Travel,Three,£48.87
4,Vagabonding: An Uncommon Guide to the Art of L...,Travel,Two,£36.94
5,Under the Tuscan Sun,Travel,Three,£37.33
...,...,...,...,...
996,Why the Right Went Wrong: Conservatism--From G...,Politics,Four,£52.65
997,Equal Is Unfair: America's Misguided Fight Aga...,Politics,One,£56.86
998,Amid the Chaos,Cultural,One,£36.58
999,Dark Notes,Erotica,Five,£19.19
